In [ ]:
!pip3 install tensorflow_text>=2.0.0rc0

In [ ]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 1.6MB/s 
     |████████████████████████████████| 7.1MB 6.4MB/s 


In [ ]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

import pymorphy2
import re
from scipy import spatial
import numpy as np
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity as cosine
import sys
import pickle

import gzip
import shutil

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('russian') + stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

In [ ]:
morph = pymorphy2.MorphAnalyzer()

In [ ]:
#Data for features

In [ ]:
dict_queries = dict()
dict_queries_NoNnorm = dict()

with open('Data/queries_b.txt','r', encoding='utf-8') as f:
    for line in f.readlines():
        tmp = line.strip().split('\t')
        if tmp[0] not in dict_queries:
            tmp_w = tmp[1].split(' ')
            norm_form =[]
            for w in tmp_w:
                norm_form.append(morph.parse(w)[0].normal_form)
            string_arr = [i for i in norm_form if i not in stop_words]
            norm_form = ' '.join(string_arr)
            dict_queries[tmp[0]] = norm_form
        if tmp[0] not in dict_queries_NoNnorm:
            dict_queries_NoNnorm[tmp[0]] = tmp[1]

dict_titles = dict()
dict_titles_NoNnorm = dict()

with open('Data/titles_norm.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        tmp = line.strip().split('\t')
        string_arr = [i for i in tmp[1].lower().split(' ') if i not in stop_words]
        dict_titles[tmp[0]] = ' '.join(string_arr)
with open('Data/titles.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        tmp = line.strip().split('\t')
        #print(tmp)
        if len(tmp) == 2:
            dict_titles_NoNnorm[tmp[0]] = tmp[1].lower()

In [ ]:
X_split = []
query_split = []
with open('Data/train.marks.tsv', 'r', encoding='utf-8') as f:
  for line in f.readlines():
    tmp = line.strip().split('\t')
    query_split.append(tmp[0])
    X_split.append(tmp[1])

with open('Data/sample.csv', 'r', encoding='utf-8') as f:
  f.readline()
  for line in f.readlines():
    tmp = line.strip().split(',')
    query_split.append(tmp[0])
    X_split.append(tmp[1])

In [ ]:
X_split = np.array(X_split)
query_split = np.array(query_split)

In [ ]:
#creating features by embedding qury/title

In [ ]:
def get_wordembedding(word):
  word_embedding = words_emb_dict[word]
  return word_embedding

In [ ]:
def get_words_sim_dict(string_q, string_t):
  res_dict = dict()
  for word_q in string_q.split(' '):
    a = get_wordembedding(word_q)
    for word_t in string_t.split(' '):
      b = get_wordembedding(word_t)
      res_dict[word_q + '\t'+ word_t] = cosine(a, b)[0][0]
  return res_dict

In [ ]:
def get_result(query, title, res_dict):
  sorted_dict = sorted(res_dict.items(), key=lambda kv: -1*kv[1])
  #print(sorted_dict)
  result = 0.0
  nums = 0.0
  for x in sorted_dict:
    words = x[0].split('\t')
    score = x[1]
    if words[0] in query and words[1] in title:
      result += score
      nums += 1
      query = query.replace(words[0], '')
      title = title.replace(words[1], '')
      #print(query, title)
    if nums == 0.0:
      return 0.0
  return result / nums

In [ ]:
j = 0
with open('USE_feat_nonnorm.txt', 'w') as f:
  for qid in np.unique(query_split):
    j+=1
    sys.stdout.write('\r'+str(j) + '/6311')
    if j == 1192: #for non norm: 1192                      #
      continue                                             #
    if j <= 1409 and j>=1404: #1404-1409 #for norm         # There is some problem (not enough RAM) with queries-titles when you try to put it all together to model 
      continue                                             # TODO: add code for embedding them individually
    if qid in ['1204', '2260', '2261', '2262']: #for norm  #
      continue                                             #
    if qid in dict_queries_NoNnorm:
      tid = X_split[np.argwhere(query_split == qid).ravel()]
      titles = [' ' if id_ not in dict_titles_NoNnorm else dict_titles_NoNnorm[id_] for id_ in tid]
      res = cosine(embed([dict_queries_NoNnorm[qid]]), embed(titles)).ravel()
      for i in range(len(tid)):
        f.write(qid+'\t'+tid[i]+'\t'+str(res[i])+'\n')

6311/6311

In [ ]:
j = 0
with open('USE_feat.txt', 'w') as f:
  for qid in np.unique(query_split):
    j+=1
    sys.stdout.write('\r'+str(j) + '/6311')
    #if j == 1192: #for non norm: 1192                     #
    #  continue                                            #
    if j <= 1409 and j>=1404: #1404-1409 #for norm         # There is some problem (not enough RAM) with queries-titles when you try to put it all together to model 
      continue                                             # TODO: add code for embedding them individually
    if qid in ['1204', '2260', '2261', '2262']: #for norm  #
      continue                                             #
    if qid in dict_queries:
      tid = X_split[np.argwhere(query_split == qid).ravel()]
      titles = [' ' if id_ not in dict_titles else dict_titles[id_] for id_ in tid]
      res = cosine(embed([dict_queries[qid]]), embed(titles)).ravel()
      for i in range(len(tid)):
        f.write(qid+'\t'+tid[i]+'\t'+str(res[i])+'\n')

6311/6311

In [ ]:
j = 0
for qid in np.unique(query_split):
  j+=1
  if j == 1192:
    print(qid)
  if j <= 1409 and j>=1404:
    print(qid)

207
2260
2261
2262
2263
2264
2265


In [ ]:
qid_bad = ['1204', '207','2260', '2261','2262','2263','2264','2265'] #for nonnorm

In [ ]:
with open('USE_feat_add.txt', 'w') as f:
  for qid in tqdm(qid_bad):
    tid = X_split[np.argwhere(query_split == qid).ravel()]
    titles = [' ' if id_ not in dict_titles else dict_titles[id_] for id_ in tid]
    vec_q = embed([dict_queries[qid]])
    for i in range(len(titles)):
      res = cosine(vec_q,embed(titles[i]))
      f.write(qid+ '\t'+tid[i]+'\t'+str(res[0][0])+'\n')
    #res = cosine(embed([dict_queries[qid]]), embed(titles)).ravel()

100%|██████████| 8/8 [00:53<00:00,  6.67s/it]


In [ ]:
with open('USE_feat_nonnorm_add.txt', 'w') as f:
  for qid in tqdm(qid_bad):
    tid = X_split[np.argwhere(query_split == qid).ravel()]
    titles = [' ' if id_ not in dict_titles_NoNnorm else dict_titles_NoNnorm[id_] for id_ in tid]
    vec_q = embed([dict_queries_NoNnorm[qid]])
    for i in range(len(titles)):
      res = cosine(vec_q,embed(titles[i]))
      f.write(qid+ '\t'+tid[i]+'\t'+str(res[0][0])+'\n')

100%|██████████| 8/8 [01:04<00:00,  8.10s/it]


In [ ]:
#creating of embedding of all words from titles+queries (normolized) + creating feature like in presentation

In [ ]:
big_string = ' '.join([dict_titles[i] for i in dict_titles]) + ' ' + ' '.join([dict_queries[i] for i in dict_queries])

i=0
words_emb_dict = dict()
for word in np.unique(big_string.split(' ')):
  sys.stdout.write('\r'+str(i) + '/181406')
  i+=1
  if word in words_emb_dict:
    continue
  words_emb_dict[word]=embed(word)

with open('words_emb_dict.pickle', 'wb') as f:
  pickle.dump(words_emb_dict, f)

In [ ]:
#because of time limit you will need to use these commands to add words emb dictionary
#with open('words_emb_dict.pickle', 'rb') as f:
#  words_emb_dict = pickle.load(f)

In [ ]:
with open('USE_feat_byword.txt','w',encoding='utf-8') as f:
  for i in range(len(X_split)):
    if (X_split[i] in dict_titles) and (query_split[i] in dict_queries):
      query = dict_queries[query_split[i]]
      title = dict_titles[X_split[i]]
      res_dict = get_words_sim_dict(query, title)
      res_ = get_result(query, title, res_dict)
      f.write(query_split[i] + '\t' + X_split[i] + '\t'+str(res_)+'\n')
    else:
       f.write(query_split[i] + '\t' + X_split[i] + '\t0.0'+'\n')
    sys.stdout.write('\r'+str(i) + '/606050')

606049/606050

In [ ]:
#wodr+word embedding for creating features

In [ ]:
def make_pair(string):
  tmp = string.split(' ')
  tmp_p = []
  for i in range(0, len(tmp)-1, 2):
    tmp_p.append(tmp[i]+' '+tmp[i+1])
  if len(tmp) % 2 == 1:
    tmp_p.append(tmp[-1])
  return tmp_p

In [ ]:
array_of_pair = []
for string in dict_titles:
  array_of_pair += make_pair(dict_titles[string])
for string in dict_queries:
  array_of_pair += make_pair(dict_queries[string])

In [ ]:
pairs_emb_dict = dict()

In [ ]:
i = 0
for pair in np.unique(array_of_pair):
  sys.stdout.write('\r'+str(i) + '/871826')
  i+=1
  if pair in pairs_emb_dict:
    continue
  pairs_emb_dict[pair]=embed(pair)

871825/871826

In [ ]:
with open('pairs_emb_dict.pickle', 'wb') as f:
  pickle.dump(pairs_emb_dict, f)

In [ ]:
#same problem
#with open('pairs_emb_dict.pickle', 'rb') as f:
#  pairs_emb_dict = pickle.load(f)

In [ ]:
def get_wordembedding(word):
  word_embedding = pairs_emb_dict[word]
  return word_embedding

In [ ]:
def get_words_sim_dict(string_q, string_t):
  res_dict = dict()
  for word_q in make_pair(string_q):
    a = get_wordembedding(word_q)
    for word_t in make_pair(string_t):
      b = get_wordembedding(word_t)
      res_dict[word_q + '\t'+ word_t] = cosine(a, b)[0][0]
  #print(res_dict)
  return res_dict

In [ ]:
def get_result(query, title, res_dict):
  sorted_dict = sorted(res_dict.items(), key=lambda kv: -1*kv[1])
  #print(sorted_dict)
  result = 0.0
  nums = 0.0
  for x in sorted_dict:
    words = x[0].split('\t')
    score = x[1]
    if words[0] in query and words[1] in title:
      result += score
      nums += 1
      query = query.replace(words[0], '', 1)
      title = title.replace(words[1], '', 1)
      #print(query, title)
    if nums == 0.0:
      return 0.0
  return result / nums

In [ ]:
with open('USE_feat_bypair.txt','w',encoding='utf-8') as f: 
  for i in range(len(X_split)): 
    if (X_split[i] in dict_titles) and (query_split[i] in dict_queries): 
      query = dict_queries[query_split[i]] 
      title = dict_titles[X_split[i]]
      #if (regex.search(query) != None) or (regex.search(title) != None):
  #  f.write(query_split[i] + '\t' + X_split[i] + '\t---'+'\n')
  #  continue
      res_dict = get_words_sim_dict(query, title)
      res_ = get_result(query, title, res_dict)
      f.write(query_split[i] + '\t' + X_split[i] + '\t'+str(res_)+'\n')
    else:
      f.write(query_split[i] + '\t' + X_split[i] + '\t0.0'+'\n')
    #break
    sys.stdout.write('\r'+str(i) + '/606050')

606049/606050